In [8]:
%pip install sumy
%pip install gensim==3.8.3
%pip install transformers
%pip install torch
%pip install tensorflow
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.
  Using cached gensim-3.8.3.tar.gz (23.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for gensim
Failed to build gensim
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [753 lines of output]
      c:\ProgramData\Anaconda3\envs\jinsu\lib\site-packages\setuptools\__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      c:\ProgramData\Anaconda3\envs\jinsu\lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      c:\ProgramData\Anaconda3\envs\jinsu\lib\site-packages\setuptools\_distutils\dist.py:26

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
# 해당 분야 상위 뉴스 HTML 가져오기
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}  # ex. 'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ...생략...'

res = requests.get(news_link, headers = headers)
print(res.text)

NameError: name 'requests' is not defined

In [2]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}  # ex. 'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ...생략...'
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

# 뉴스의 기본 정보 가져오기
# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    # 임시 이미지
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
                + "&sid1=" \
                + sid
    print("section url : ", sec_url)
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)

    # 해당 분야 상위 뉴스 3개 가져오기
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img
        }
        news_list3.append(news_info)
        
    return news_list3

# 뉴스 본문 가져오기
def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', class_="_article_body_contents")

    news_contents = ''
    for content in body:
        if type(content) is bs4.element.NavigableString and len(content) > 50:
            # content.strip() : whitepace 제거 (참고 : https://www.tutorialspoint.com/python3/string_strip.htm)
            # 뉴스 요약을 위하여 '.' 마침표 뒤에 한칸을 띄워 문장을 구분하도록 함
            news_contents += content.strip() + ' '
        
    return news_contents
        
    
# '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    # 뉴스 결과를 담아낼 dictionary
    news_dic = dict()
    
    # sections : '정치', '경제', '사회'
    sections = ["pol", "eco","soc"]
    # section_ids :  URL에 사용될 뉴스  각 부문 ID
    section_ids = ["100", "101","102"]
    
    for sec, sid in zip(sections, section_ids):   
        # 뉴스의 기본 정보 가져오기
        news_info = get_top3_news_info(sec, sid)
        #print(news_info)
        for news in news_info:
            # 뉴스 본문 가져오기
            news_url = news['news_url']
            news_contents = get_news_contents(news_url)
            
            # 뉴스 정보를 저장하는 dictionary를 구성
            news['news_contents'] = news_contents

        news_dic[sec] = news_info
    
    return news_dic
    
# 함수 호출 - '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
news_dic = get_naver_news_top3()
# 경제의 첫번째 결과 확인하기 
news_dic['eco'][0]

section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100


TypeError: 'NoneType' object is not iterable

수정된 코드

In [3]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    
    return soup

# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = f"https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={sid}"
    print("section url : ", sec_url)
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)

    # 해당 분야 상위 뉴스 3개 가져오기
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_info = {
            "title": li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t", "").replace("\r", ""),
            "date": li.find(class_="date").text,
            "news_url": li.a.attrs.get('href'),
            "image_url": li.img.attrs.get('src') if li.img else default_img
        }
        news_list3.append(news_info)
        
    return news_list3

# 뉴스 본문 가져오기
def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', class_="_article_body_contents")

    news_contents = ''
    if body is not None:  # body가 None이 아닐 경우에만 반복문 실행
        for content in body:
            if isinstance(content, bs4.element.NavigableString) and len(content) > 50:
                news_contents += content.strip() + ' '
    else:
        print("뉴스 본문을 찾을 수 없습니다.")  # 디버깅용 메시지

    return news_contents

# '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    news_dic = dict()
    
    sections = ["pol", "eco", "soc"]
    section_ids = ["100", "101", "102"]
    
    for sec, sid in zip(sections, section_ids):   
        news_info = get_top3_news_info(sec, sid)
        for news in news_info:
            news_url = news['news_url']
            news_contents = get_news_contents(news_url)
            news['news_contents'] = news_contents

        news_dic[sec] = news_info
    
    return news_dic
    
# 함수 호출 - '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
news_dic = get_naver_news_top3()
# 경제의 첫번째 결과 확인하기 
print(news_dic['eco'][0])


section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100
뉴스 본문을 찾을 수 없습니다.
뉴스 본문을 찾을 수 없습니다.
뉴스 본문을 찾을 수 없습니다.
section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101
뉴스 본문을 찾을 수 없습니다.
뉴스 본문을 찾을 수 없습니다.
뉴스 본문을 찾을 수 없습니다.
section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=102
뉴스 본문을 찾을 수 없습니다.
뉴스 본문을 찾을 수 없습니다.
뉴스 본문을 찾을 수 없습니다.
{'title': '한수원, 디지털범죄 취약층에 개인정보보호 캠페인', 'date': '1분전', 'news_url': 'https://n.news.naver.com/mnews/article/014/0005248517?sid=101', 'image_url': 'https://imgnews.pstatic.net/image/origin/014/2024/10/02/5248517.jpg?type=nf106_72', 'news_contents': ''}


In [15]:
#base코드 삭제 ㄴㄴ 
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
for sid in ['100']:#, '101', '102']:
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
                + "&sid1=" \
                + sid
    print("section url : ", sec_url)
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)

    # 해당 분야 상위 뉴스 3개 가져오기
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img
        }
        print(news_info)

section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100
{'title': '국민의힘 "이재명 방탄·국민 호도 청문회"', 'date': '1분전', 'news_url': 'https://n.news.naver.com/mnews/article/057/0001844988?sid=100', 'image_url': 'https://imgnews.pstatic.net/image/origin/057/2024/10/02/1844988.jpg?type=ofullfill106_72'}
{'title': '정부, 중동 정세 악화 우려 표명…“외교적 해법 모색해야”', 'date': '2분전', 'news_url': 'https://n.news.naver.com/mnews/article/277/0005479383?sid=100', 'image_url': 'https://imgnews.pstatic.net/image/origin/277/2024/10/02/5479383.jpg?type=ofullfill106_72'}
{'title': '與 “이재명에 보고했다 자백” vs 이화영 “허위 진술”', 'date': '2분전', 'news_url': 'https://n.news.naver.com/mnews/article/449/0000287064?sid=100', 'image_url': 'https://imgnews.pstatic.net/image/origin/449/2024/10/02/287064.jpg?type=ofullfill106_72'}


In [20]:
#지금 코드수정중인 곳
import requests
from bs4 import BeautifulSoup
import bs4.element

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    # 임시 이미지
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"

    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" + "&sid1=" + sid
    print("section url: ", sec_url)

    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)

    # 해당 분야 상위 뉴스 3개 가져오기
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline')
    if lis3:
        lis3 = lis3.find_all("li", limit=3)
        for li in lis3:
            # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
            news_info = {
                "title": li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t", "").replace("\r", ""),
                "date": li.find(class_="date").text,
                "news_url": li.a.attrs.get('href'),
                "image_url": li.img.attrs.get('src') if li.img else default_img
            }
            news_list3.append(news_info)

    return news_list3

# 뉴스 본문 가져오기
def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', id="articleBodyContents")

    news_contents = ''
    if body:
        for content in body:
            if isinstance(content, bs4.element.NavigableString) and len(content.strip()) > 50:
                # 뉴스 요약을 위하여 '.' 마침표 뒤에 한칸을 띄워 문장을 구분하도록 함
                news_contents += content.strip() + ' '

    return news_contents

# 정치, 경제, 사회 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    news_dic = dict()

    sections = ["pol", "eco", "soc"]
    section_ids = ["100", "101", "102"]

    for sec, sid in zip(sections, section_ids):
        news_info = get_top3_news_info(sec, sid)
        for news in news_info:
            news_url = news['news_url']
            news_contents = get_news_contents(news_url)
            news['news_contents'] = news_contents

        news_dic[sec] = news_info

    return news_dic

# 함수 호출 - '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
news_dic = get_naver_news_top3()
# 경제의 첫번째 결과 확인하기
print(news_dic['eco'][0])


section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100
section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101
section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=102
{'title': '국립인천해양박물관, 10월의 해양유물로 ‘포르톨라노 해도’ 선정', 'date': '2분전', 'news_url': 'https://n.news.naver.com/mnews/article/082/0001291097?sid=101', 'image_url': 'https://imgnews.pstatic.net/image/origin/082/2024/10/02/1291097.jpg?type=nf106_72', 'news_contents': ''}


In [22]:
import requests
from bs4 import BeautifulSoup
import bs4.element
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    sec_url = f"https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={sid}"
    print("section url: ", sec_url)
    
    soup = get_soup_obj(sec_url)
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline')
    if lis3:
        lis3 = lis3.find_all("li", limit=3)
        for li in lis3:
            news_info = {
                "title": li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t", "").replace("\r", ""),
                "date": li.find(class_="date").text,
                "news_url": li.a.attrs.get('href'),
                "image_url": li.img.attrs.get('src') if li.img else default_img
            }
            news_list3.append(news_info)
    return news_list3

# 뉴스 본문 가져오기
def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', id="articleBodyContents")

    news_contents = ''
    if body:
        for content in body:
            if isinstance(content, bs4.element.NavigableString) and len(content.strip()) > 50:
                news_contents += content.strip() + ' '

    return news_contents

# 정치, 경제, 사회 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    news_dic = dict()
    sections = ["pol", "eco", "soc"]
    section_ids = ["100", "101", "102"]

    for sec, sid in zip(sections, section_ids):
        news_info = get_top3_news_info(sec, sid)
        for news in news_info:
            news_url = news['news_url']
            news_contents = get_news_contents(news_url)
            news['news_contents'] = news_contents
        news_dic[sec] = news_info

    return news_dic

# KoBART 모델과 토크나이저 초기화
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

# 뉴스 요약하기
def summarize_news(news_list):
    for news_info in news_list:
        if news_info['news_contents']:
            inputs = tokenizer(news_info['news_contents'], max_length=1024, return_tensors='pt', truncation=True)
            summary_ids = model.generate(inputs['input_ids'], max_length=50, min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)
            snews_contents = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            news_info['snews_contents'] = snews_contents
        else:
            news_info['snews_contents'] = "내용이 없습니다."

# 함수 호출 - '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
news_dic = get_naver_news_top3()

# 요약 실행
summarize_news(news_dic['eco'])

# 요약 결과 출력 - 첫번째 뉴스
print("==== 첫번째 뉴스 원문 ====")
print(news_dic['eco'][0]['news_contents'])
print("\n==== 첫번째 뉴스 요약문 ====")
print(news_dic['eco'][0]['snews_contents'])

# 요약 결과 출력 - 두번째 뉴스
print("==== 두번째 뉴스 원문 ====")
print(news_dic['eco'][1]['news_contents'])
print("\n==== 두번째 뉴스 요약문 ====")
print(news_dic['eco'][1]['snews_contents'])


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100
section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101
section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=102
==== 첫번째 뉴스 원문 ====


==== 첫번째 뉴스 요약문 ====
내용이 없습니다.
==== 두번째 뉴스 원문 ====


==== 두번째 뉴스 요약문 ====
내용이 없습니다.


In [16]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# KoBART 모델과 토크나이저 초기화
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

# 섹션 지정
my_section = 'eco'
news_list3 = news_dic[my_section]

# 뉴스 요약하기
for news_info in news_list3:
    # 뉴스 내용 확인
    print("뉴스 내용:", news_info['news_contents'])  # 뉴스 내용을 출력하여 확인
    if not news_info['news_contents']:  # 뉴스 내용이 비어 있는지 확인
        print("뉴스 내용이 비어 있습니다.")
        news_info['snews_contents'] = "내용이 없습니다."  # 비어 있을 경우 대체 텍스트
        continue
    
    try:
        # 뉴스 내용을 토크나이징하고 요약을 생성합니다.
        inputs = tokenizer(news_info['news_contents'], max_length=1024, return_tensors='pt', truncation=True)
        summary_ids = model.generate(inputs['input_ids'], max_length=50, min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)
        snews_contents = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    except Exception as e:
        print(f"Error: {e}")  # 오류 발생 시 출력
        snews_contents = None

    if not snews_contents:
        # 요약에 실패하거나 내용이 너무 짧을 경우 첫 3개의 문장을 가져옵니다.
        news_sentences = news_info['news_contents'].split('.')
        if len(news_sentences) > 3:
            snews_contents = '.'.join(news_sentences[:3])
        else:
            snews_contents = '.'.join(news_sentences)

    news_info['snews_contents'] = snews_contents

## 요약 결과 - 첫번째 뉴스
print("==== 첫번째 뉴스 원문 ====")
print(news_list3[0]['news_contents'])
print("\n==== 첫번째 뉴스 요약문 ====")
print(news_list3[0]['snews_contents'])

## 요약 결과 - 두번째 뉴스
print("==== 두번째 뉴스 원문 ====")
print(news_list3[1]['news_contents'])
print("\n==== 두번째 뉴스 요약문 ====")
print(news_list3[1]['snews_contents'])


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


뉴스 내용: 
뉴스 내용이 비어 있습니다.
뉴스 내용: 
뉴스 내용이 비어 있습니다.
뉴스 내용: 
뉴스 내용이 비어 있습니다.
==== 첫번째 뉴스 원문 ====


==== 첫번째 뉴스 요약문 ====
내용이 없습니다.
==== 두번째 뉴스 원문 ====


==== 두번째 뉴스 요약문 ====
내용이 없습니다.


In [23]:
import requests
from bs4 import BeautifulSoup
import bs4.element
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    sec_url = f"https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={sid}"
    print("section url: ", sec_url)
    
    soup = get_soup_obj(sec_url)
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline')
    if lis3:
        lis3 = lis3.find_all("li", limit=3)
        for li in lis3:
            news_info = {
                "title": li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t", "").replace("\r", ""),
                "date": li.find(class_="date").text,
                "news_url": li.a.attrs.get('href'),
                "image_url": li.img.attrs.get('src') if li.img else default_img
            }
            news_list3.append(news_info)
    return news_list3

# 뉴스 본문 가져오기
def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', id="articleBodyContents")  # 또는 적절한 클래스 이름으로 수정

    news_contents = ''
    if body:
        news_contents = body.get_text(strip=True, separator=' ')  # 본문 내용 가져오기
    else:
        print(f"No body found for: {url}")  # 본문이 없을 경우 출력
    
    return news_contents

# 정치, 경제, 사회 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    news_dic = dict()
    sections = ["pol", "eco", "soc"]
    section_ids = ["100", "101", "102"]

    for sec, sid in zip(sections, section_ids):
        news_info = get_top3_news_info(sec, sid)
        for news in news_info:
            news_url = news['news_url']
            news_contents = get_news_contents(news_url)
            news['news_contents'] = news_contents
        news_dic[sec] = news_info

    return news_dic

# KoBART 모델과 토크나이저 초기화
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

# 뉴스 요약하기
def summarize_news(news_list):
    for news_info in news_list:
        if news_info['news_contents']:
            inputs = tokenizer(news_info['news_contents'], max_length=1024, return_tensors='pt', truncation=True)
            summary_ids = model.generate(inputs['input_ids'], max_length=50, min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)
            snews_contents = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            news_info['snews_contents'] = snews_contents
        else:
            news_info['snews_contents'] = "내용이 없습니다."

# 함수 호출 - '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
news_dic = get_naver_news_top3()

# 요약 실행
summarize_news(news_dic['eco'])

# 요약 결과 출력 - 첫번째 뉴스
print("==== 첫번째 뉴스 원문 ====")
print(news_dic['eco'][0]['news_contents'])
print("\n==== 첫번째 뉴스 요약문 ====")
print(news_dic['eco'][0]['snews_contents'])

# 요약 결과 출력 - 두번째 뉴스
print("==== 두번째 뉴스 원문 ====")
print(news_dic['eco'][1]['news_contents'])
print("\n==== 두번째 뉴스 요약문 ====")
print(news_dic['eco'][1]['snews_contents'])


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100
No body found for: https://n.news.naver.com/mnews/article/003/0012817133?sid=100
No body found for: https://n.news.naver.com/mnews/article/028/0002709716?sid=100
No body found for: https://n.news.naver.com/mnews/article/032/0003324058?sid=100
section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101
No body found for: https://n.news.naver.com/mnews/article/082/0001291097?sid=101
No body found for: https://n.news.naver.com/mnews/article/119/0002877867?sid=101
No body found for: https://n.news.naver.com/mnews/article/018/0005848908?sid=101
section url:  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=102
No body found for: https://n.news.naver.com/mnews/article/023/0003861854?sid=102
No body found for: https://n.news.naver.com/mnews/article/009/0005373582?sid=102
No body found for: https://n.news.naver.com/mnews/article/016/0002369570?sid=102
==== 첫번째 뉴스 원문 ====


==== 첫번째 뉴스 요약문 ==